```
0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
1 - the id of the tweet (2087)
2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)
3 - the query (lyx). If there is no query, then this value is NO_QUERY.
4 - the user that tweeted (robotickilldozr)
5 - the text of the tweet (Lyx is cool)
```

## Downloading data
There is a script located under data/sentiment140/download.sh

In [12]:
import os
import pandas as pd

csv_path = os.path.join(os.getcwd(), "..", "data", "sentiment140")
train_path = os.path.join(csv_path, "train.csv")
test_path = os.path.join(csv_path, "test.csv")

columns = ["sentiment", "id", "date", "query", "user", "text"]
train_df = pd.read_csv(train_path, header=None, names=columns, encoding="latin1")

In [13]:
train_df

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [14]:
# we're only interested in sentiment and text
train_df = train_df[["sentiment", "text"]]
train_df

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [15]:
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
import spacy
import re

STOP = stopwords.words("english")

twitter_words_to_filter = ["rt"]
def filter_single_letter(word):
    return len(word) > 1 or word == 'a' or word == 'i'

tt = TweetTokenizer()
def twitter_preprocessing(post):
    tokenized_post = tt.tokenize(post)
    pattern = re.compile(r"^[a-zA-Z\!\?\,\.\']+$")
    dot_pattern = re.compile(r"\.{2,}")
    url_matcher = re.compile(r"^[a-zA-Z]+\.[a-zA-Z]+$")
    tokens = [
        w.lower() for w in tokenized_post
        if pattern.match(w) and not dot_pattern.match(w)
        and w.lower() not in twitter_words_to_filter
        and filter_single_letter(w.lower())
        and not url_matcher.match(w.lower())
    ]
    return tokens

In [9]:
import psutil
total_gb_ram = int(psutil.virtual_memory().total / 1024 / 1024 / 1024)
print(f"{total_gb_ram} GB RAM")

32 GB RAM


In [16]:
# apply preprocessing to the tweets
# (depending on computational power)
if total_gb_ram >= 16:
    import mapply
    mapply.init()

    train_df["text"] = train_df.text.mapply(twitter_preprocessing, train_df["text"].tolist())
else:
    train_df["text"] = train_df["text"].apply(twitter_preprocessing)

  0%|          | 0/80 [00:00<?, ?it/s]

/var/folders/0k/1bg63zt532nb9d86g5tk_6vh0000gn/T/ipykernel_6480/4144196185.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["text"] = train_df.text.mapply(twitter_preprocessing, train_df["text"].tolist())


In [17]:
train_df

,sentiment,text
0,0,"[awww, that's, a, bummer, you, shoulda, got, d..."
1,0,"[is, upset, that, he, can't, update, his, face..."
2,0,"[i, dived, many, times, for, the, ball, manage..."
3,0,"[my, whole, body, feels, itchy, and, like, its..."
4,0,"[no, it's, not, behaving, at, all, i'm, mad, w..."
...,...,...
1599995,4,"[just, woke, up, having, no, school, is, the, ..."
1599996,4,"[very, cool, to, hear, old, walt, interviews]"
1599997,4,"[are, you, ready, for, your, mojo, makeover, a..."
1599998,4,"[happy, birthday, to, my, boo, of, alll, time,..."


In [18]:
# filter out texts with less than 5 words:
train_df = train_df[train_df.text.map(len) > 5]

In [19]:
# add a column that is the text as a string:
train_df["text_str"] = train_df.text.map(lambda x: " ".join(x))

/var/folders/0k/1bg63zt532nb9d86g5tk_6vh0000gn/T/ipykernel_6480/1118835850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["text_str"] = train_df.text.map(lambda x: " ".join(x))


In [20]:
train_df = train_df.rename(columns={"text": "tokens", "text_str": "text"})

In [21]:
train_df

,sentiment,tokens,text
0,0,"[awww, that's, a, bummer, you, shoulda, got, d...",awww that's a bummer you shoulda got david car...
1,0,"[is, upset, that, he, can't, update, his, face...",is upset that he can't update his facebook by ...
2,0,"[i, dived, many, times, for, the, ball, manage...",i dived many times for the ball managed to sav...
3,0,"[my, whole, body, feels, itchy, and, like, its...",my whole body feels itchy and like its on fire
4,0,"[no, it's, not, behaving, at, all, i'm, mad, w...",no it's not behaving at all i'm mad why am i h...
...,...,...,...
1599994,4,"[yeah, that, does, work, better, than, just, w...",yeah that does work better than just waiting f...
1599995,4,"[just, woke, up, having, no, school, is, the, ...",just woke up having no school is the best feel...
1599996,4,"[very, cool, to, hear, old, walt, interviews]",very cool to hear old walt interviews
1599997,4,"[are, you, ready, for, your, mojo, makeover, a...",are you ready for your mojo makeover ask me fo...


In [22]:
save_path = os.path.join(csv_path, "processed_train.csv")
train_df.to_csv(save_path, index=False)